In [25]:
def dudc(P,Q):
    P.sort()
    Q.sort()
    I = []
    nb_p = 0
    if len(Q) == 0 : 
        print("Q is empty set")
        return I
    elif len(Q)==1 : 
        for p in P : 
            if Q[0] <= p[0] and p[0] <= Q[1] : nb_p += 1
        if nb_p == len(P) : return Q
    else : 
        for p in P :
            for i in range(len(Q)):
                if i == len(Q)-1 :
                    if Q[i][0] <= p[0] and p[0] <= Q[i][1] : 
                        nb_p += 1
                        I.append(Q[len(Q)-1])
                elif p[0] <= Q[i][1] and p[0] < Q[i+1][0] : 
                    nb_p += 1 
                    I.append(Q[i])
                    break
            
        if nb_p == len(P) : return I
        else :
            print("We can't cover all the points")
            return I

In [26]:
P = [(1.5,0), (2.5,0), (1.8,0), (0.3,0), (2.54,0)]
Q = [(1.2,1.6),(1.6,2),(1.8,2.2),(2.5,2.9),(0,0.4),(1.4,1.8),(0.01,0.41)]
print("Interval = ", dudc(P,Q))

Interval =  [(0.01, 0.41), (1.4, 1.8), (1.8, 2.2), (2.5, 2.9), (2.5, 2.9)]
